## This is the test for bi-lstm, lstm and cnn models


In [ ]:
import numpy as np
import pandas as pd
import time
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import time


ID_COLUMN = 'id'
TEXT_COLUMN = 'comment_text'
TARGET_COLUMN = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


read data


In [2]:
train_data = pd.read_csv('./output/eda_train.csv', encoding='utf-8')
train_labels = train_data[TARGET_COLUMN].values
test_data = pd.read_csv('./input/test.csv', encoding='utf-8')


load GloVe 

In [3]:
with open('./glove.840B.300d.txt', encoding='utf-8') as f:
    embedding_values = {}
    for line in tqdm(f):
        value = line.split(' ')
        word = value[0]
        coef = np.array(value[1:], dtype='float32')
        embedding_values[word] = coef
        
all_embs = np.stack(embedding_values.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()
emb_mean, emb_std

# mapping text to index
train_text = train_data['comment_text']
token = Tokenizer(num_words=20000)
token.fit_on_texts(train_text)
train_seq = token.texts_to_sequences(train_text)
train_pad_seq = pad_sequences(train_seq, maxlen=300)

test_text = test_data['comment_text']
test_seq = token.texts_to_sequences(test_text)
test_pad_seq = pad_sequences(test_seq, maxlen=300)

vocab_size = len(token.word_index)+1
print(vocab_size)

# get embedding_matrix ready
embedding_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, 300))
for word, i in tqdm(token.word_index.items()):
    values = embedding_values.get(word)
    if values is not None:
        embedding_matrix[i] = values
        

2196017it [02:54, 12552.41it/s]
100%|██████████| 182957/182957 [00:01<00:00, 123281.36it/s]


182958


models


In [ ]:
from bi_lstm import bilstm_model
model = bilstm_model(vocab_size=vocab_size, embedding_matrix=embedding_matrix)
model.fit(train_pad_seq, train_labels, epochs=5, batch_size=32, validation_split=0.1)
model.summary()
model.save_weights('./output/bi-lstm_model.h5')
predict = model.predict(test_pad_seq)
    

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 718069 samples, validate on 79786 samples
Epoch 1/5


final


In [ ]:
sample_submission = pd.read_csv('./input/sample_submission.csv', encoding='utf-8')
sample_submission[TARGET_COLUMN] = predict
sample_submission.to_csv('./output/eda_bi-lstm_submission.csv', index=False)
print('finished')
        
